# はじめに

このノートブックはkaggleに参加してまもない私にとって初めてのデータ分析を行ったノートになります。

ノートの中にはたびたび冗長なソースコードを目にする可能性があると思いますのでご理解いただけると嬉しいです。

※もしもっと良い書き方があればぜひ教えてください！

# このノートでわかること

- コンペの目的
- 提供データの概要
- 提供データの基礎分析内容

# コンペの目的を把握する

「M5 Forecasting - Accuracy」では、アメリカで最大の小売店であるWalmartの28日間の売り上げを予測します。

# 提供されたデータを理解する

- sales_train_validation.csv

製品および店舗ごとの日次販売台数データです。「いつ、どこの店舗で、どのカテゴリの、どの商品が売れたのか」を確認することが可能です。

- calendar.csv 

製品の販売日に関する情報が含まれています。「d_」で「sales_train_validation.csv」と結合することにより、
日付ありのsales_train_validation.csvのデータを確認することが可能です。

- sample_submission.csv

提出の正しい形式を示しています。

- sell_prices.csv

各商品の販売価格が示されています。

- sales_train_evaluation.csv

締め切りの1か月前に入手可能。 
売上高が含まれます。

今回の分析では「sales_train_validation.csv」と「calendar.csv」を主に使用します。



# データの読み込みと提供データの確認

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from itertools import cycle
# set_option
# 50行まで表示する。max_columnsはすべてのカラムを表示することを指定している。
pd.set_option('max_columns', 50)

# 格好いいグラフ描画をするための下準備
# https://qiita.com/eriksoon/items/b93030ba4dc686ecfbba
plt.style.use('bmh')

# inputディレクトに配置したデータの確認
!ls -GFlash --color ../input/m5-forecasting-accuracy/

In [ ]:
# データの読み込み
INPUT_DIR = '../input/m5-forecasting-accuracy'
cal = pd.read_csv(f'{INPUT_DIR}/calendar.csv')
stv = pd.read_csv(f'{INPUT_DIR}/sales_train_validation.csv')
ss = pd.read_csv(f'{INPUT_DIR}/sample_submission.csv')
sellp = pd.read_csv(f'{INPUT_DIR}/sell_prices.csv')

In [ ]:
# 提出形式の確認
ss.head()

In [ ]:
# トレーニングデータの形式確認
stv.head()

In [ ]:
# カレンダーデータの確認
cal.head()

# データ分析

## 分析内容
1. 時系列で商品カテゴリごと販売数（全年度・年次・週次）
2. 時系列で店舗ごとに販売数（全年度・年次・週次）
3. 時系列で地域ごとに販売数（全年度・年次・週次）
4. 地域毎の店舗数 
5. 地域ごとに商品カテゴリの販売数
6. よく売れている商品TOP5

## 下準備

In [ ]:
# トレーニングデータの大きさを確認
stv.info()

In [ ]:
# トレーニングデータの基礎分析
stv.describe() 

## 1.時系列で商品カテゴリごと販売数を確認

In [ ]:
d_cols = [c for c in stv.columns if 'd_' in c] 

In [ ]:
df= stv.groupby('cat_id')[d_cols].sum().sum(axis=1).sort_values()
df.name = 'Category_sum'
df

### カテゴリ毎の総合販売数(全年度)

In [ ]:
pd.DataFrame(df).plot(kind='barh', figsize=(15, 5), title='Total sales by category (all years)')
plt.show()

### 考察
- 食品の売り上げ販売数が多数を占めていることがわかる。

### 時系列　カテゴリ毎の総販売数　(全年度)

In [ ]:
stv.groupby('cat_id')[d_cols].sum().T.merge(cal.set_index('d')['date'],
           left_index=True,
           right_index=True,
            validate='1:1') \
    .set_index('date').plot(kind='line',figsize=(18, 8))
plt.title(" Time series Total sales by category (all years)")
plt.show()

### 考察
- 一年に一回売り上げが0になるポイントが発生している。
- 細かな周期的な変動がみられるが上記のグラフでは周期間隔が不明のため。


### 時系列　カテゴリ毎の総販売数　移動平均線　(全年度)

In [ ]:
stv.groupby('cat_id')[d_cols].sum().T.rolling(90).mean().merge(cal.set_index('d')['date'],
           left_index=True,
           right_index=True,
            validate='1:1') \
    .set_index('date').plot(kind='line',figsize=(15, 5))
plt.title("Rolling 90 Day Average Total Sales (Category)")
plt.show()

### 考察
- 全体的に緩やかなペースで売り上げが上がっている。
- 特に2013-2-17では全てのカテゴリで上昇しているように見える。

### 時系列　カテゴリ毎の総販売数　(2015年分)

2015年の情報を対象に分析を行う。

In [ ]:
df = stv.groupby('cat_id')[d_cols].sum().T.merge(cal.set_index('d')['date'],left_index=True,right_index=True,validate='1:1').set_index('date')
df[df.index.str.startswith('2015-')].plot(kind='line',figsize=(18, 8))
plt.title("Time series Total sales by category (2015)")
plt.show()

### 時系列　カテゴリ毎の総販売数　(2015年1月-3月分)

In [ ]:
df = stv.groupby('cat_id')[d_cols].sum().T.merge(cal.set_index('d')['date'],left_index=True,right_index=True,validate='1:1').set_index('date')
df[df.index.str.startswith('2015-01-') | df.index.str.startswith('2015-02-')| df.index.str.startswith('2015-03-')].plot(kind='line',figsize=(18, 8))
plt.title("Time Series Total Sales by Category (January-March 2015)")
plt.show()

### 考察
- 1月1日は正月休みの影響で売り上げが一時的に下がっている。

### 時系列　カテゴリ毎の総販売数　(2015年4月-6月分)

In [ ]:
df = stv.groupby('cat_id')[d_cols].sum().T.merge(cal.set_index('d')['date'],left_index=True,right_index=True,validate='1:1').set_index('date')
df[df.index.str.startswith('2015-04-') | df.index.str.startswith('2015-05-')| df.index.str.startswith('2015-06-')].plot(kind='line',figsize=(18, 8))
plt.title("Time Series Total Sales by Category (April - June 2015)")
plt.show()

### 時系列　カテゴリ毎の総販売数　(2015年7月-9月分)

In [ ]:
df = stv.groupby('cat_id')[d_cols].sum().T.merge(cal.set_index('d')['date'],left_index=True,right_index=True,validate='1:1').set_index('date')
df[df.index.str.startswith('2015-07-') | df.index.str.startswith('2015-08-')| df.index.str.startswith('2015-09-')].plot(kind='line',figsize=(18, 8))
plt.title("Time Series Total Sales by Category (July-September 2015)")
plt.show()

### 時系列　カテゴリ毎の総販売数　(2015年10月-12月分)

In [ ]:
df = stv.groupby('cat_id')[d_cols].sum().T.merge(cal.set_index('d')['date'],left_index=True,right_index=True,validate='1:1').set_index('date')
df[df.index.str.startswith('2015-10-') | df.index.str.startswith('2015-11-')| df.index.str.startswith('2015-12-')].plot(kind='line',figsize=(18, 8))
plt.title("Time Series Total Sales by Category (October - December 2015)")
plt.show()

### 時系列　カテゴリ毎の総販売数　(2015年10月)
10月のデータを確認し、週次のデータ周期を確認する。

In [ ]:
df = stv.groupby('cat_id')[d_cols].sum().T.merge(cal.set_index('d')['date'],left_index=True,right_index=True,validate='1:1').set_index('date')
df = df[df.index.str.startswith('2015-10-')].plot(kind='line',figsize=(18, 8))

plt.title("Time series Total sales by category (October 2015)")
plt.show()

### 考察
- 休日・祝日になると売り上げが上がっている。

## 2.時系列で店舗ごとに販売数

### 店舗毎の総販売数　(全年度)

In [ ]:
df= stv.groupby('store_id')[d_cols].sum().sum(axis=1).sort_values()
df.name = 'store_sum'
df

In [ ]:
pd.DataFrame(df).plot(kind='barh', figsize=(15, 5), title='Total sales per store (all years)')
plt.show()

### 時系列　店舗ごとに総販売数　(全年度)

In [ ]:
#Sales trends of each category
stv.groupby('store_id')[d_cols].sum().T.merge(cal.set_index('d')['date'],
           left_index=True,
           right_index=True,
            validate='1:1') \
    .set_index('date').plot(kind='line',figsize=(18, 8))
plt.title("Time series Total sales by store (all years)")
plt.show()

### 時系列　店舗ごとに総販売数　移動平均線　(全年度)

In [ ]:
#Rolling 90 Day Average Total Sales (Category)
stv.groupby('store_id')[d_cols].sum().T.rolling(90).mean().merge(cal.set_index('d')['date'],
           left_index=True,
           right_index=True,
            validate='1:1') \
    .set_index('date').plot(kind='line',figsize=(15, 5))
plt.title("Time series Total sales per store Moving average line (all years)")
plt.show()

### 時系列　店舗ごとに総販売数　(2015年01月-03月分)

In [ ]:
df = stv.groupby('store_id')[d_cols].sum().T.merge(cal.set_index('d')['date'],left_index=True,right_index=True,validate='1:1').set_index('date')
df[df.index.str.startswith('2015-01-') | df.index.str.startswith('2015-02-')| df.index.str.startswith('2015-03-')].plot(kind='line',figsize=(18, 8))
plt.title("Time series Total sales by store (Jan-Mar 2015)")
plt.show()

### 時系列　店舗ごとに総販売数　(2015年04月-06月分)

In [ ]:
df = stv.groupby('store_id')[d_cols].sum().T.merge(cal.set_index('d')['date'],left_index=True,right_index=True,validate='1:1').set_index('date')
df[df.index.str.startswith('2015-04-') | df.index.str.startswith('2015-05-')| df.index.str.startswith('2015-06-')].plot(kind='line',figsize=(18, 8))
plt.title("Time series Total sales by store (April - June 2015)")
plt.show()

### 時系列　店舗ごとに総販売数　(2015年07月-09月分)

In [ ]:
df = stv.groupby('store_id')[d_cols].sum().T.merge(cal.set_index('d')['date'],left_index=True,right_index=True,validate='1:1').set_index('date')
df[df.index.str.startswith('2015-07-') | df.index.str.startswith('2015-08-')| df.index.str.startswith('2015-09-')].plot(kind='line',figsize=(18, 8))
plt.title("Time series Total sales by store (Jul-Sep 2015)")
plt.show()

### 時系列　店舗ごとに総販売数　(2015年10月-12月分)

In [ ]:
df = stv.groupby('store_id')[d_cols].sum().T.merge(cal.set_index('d')['date'],left_index=True,right_index=True,validate='1:1').set_index('date')
df[df.index.str.startswith('2015-10-') | df.index.str.startswith('2015-11-')| df.index.str.startswith('2015-12-')].plot(kind='line',figsize=(18, 8))
plt.title("Time series Total sales by store (October - December 2015)")
plt.show()

### 時系列　店舗ごとに総販売数　(2015年10月分)

In [ ]:
df = stv.groupby('store_id')[d_cols].sum().T.merge(cal.set_index('d')['date'],left_index=True,right_index=True,validate='1:1').set_index('date')
df = df[df.index.str.startswith('2015-10-')].plot(kind='line',figsize=(18, 8))

plt.title("Time series Total sales by store (October 2015)")
plt.show()

## 3.【時系列】地域ごとの販売数

### 地域ごとの総販売数(全年度)

In [ ]:
df= stv.groupby('state_id')[d_cols].sum().sum(axis=1).sort_values()
df.name = 'state_sum'
df

In [ ]:
pd.DataFrame(df).plot(kind='barh', figsize=(15, 5), title='Total sales by region (all years)')
plt.show()

### 【時系列】地域ごとの総販売数 （全年度）

In [ ]:
#Sales trends of each category
stv.groupby('state_id')[d_cols].sum().T.merge(cal.set_index('d')['date'],
           left_index=True,
           right_index=True,
            validate='1:1') \
    .set_index('date').plot(kind='line',figsize=(18, 8))
plt.title("Total sales by region (all years)")
plt.show()

### 【時系列】地域ごとの総販売数 移動平均線（全年度）

In [ ]:
#Rolling 90 Day Average Total Sales (Category)
stv.groupby('state_id')[d_cols].sum().T.rolling(90).mean().merge(cal.set_index('d')['date'],
           left_index=True,
           right_index=True,
            validate='1:1') \
    .set_index('date').plot(kind='line',figsize=(15, 5))
plt.title("Time series] Total sales by region Moving average (all years)")
plt.show()

### 【時系列】地域ごとの総販売数 (2015年01月-03月分)

In [ ]:
df = stv.groupby('state_id')[d_cols].sum().T.merge(cal.set_index('d')['date'],left_index=True,right_index=True,validate='1:1').set_index('date')
df[df.index.str.startswith('2015-01-') | df.index.str.startswith('2015-02-')| df.index.str.startswith('2015-03-')].plot(kind='line',figsize=(18, 8))
plt.title("Time series] Total sales by region (Jan-Mar 2015)")
plt.show()

### 【時系列】地域ごとの総販売数 (2015年04月-06月分)

In [ ]:
df = stv.groupby('state_id')[d_cols].sum().T.merge(cal.set_index('d')['date'],left_index=True,right_index=True,validate='1:1').set_index('date')
df[df.index.str.startswith('2015-04-') | df.index.str.startswith('2015-05-')| df.index.str.startswith('2015-06-')].plot(kind='line',figsize=(18, 8))
plt.title("Time series] Total sales by region (Apr. 2015 - Jun. 2015)")
plt.show()

### 【時系列】地域ごとの総販売数 (2015年07月-09月分)

In [ ]:
df = stv.groupby('state_id')[d_cols].sum().T.merge(cal.set_index('d')['date'],left_index=True,right_index=True,validate='1:1').set_index('date')
df[df.index.str.startswith('2015-07-') | df.index.str.startswith('2015-08-')| df.index.str.startswith('2015-09-')].plot(kind='line',figsize=(18, 8))
plt.title("Time series] Total sales by region (Jul-Sep 2015)")
plt.show()

### 【時系列】地域ごとの総販売数 (2015年10月-12月分)

In [ ]:
df = stv.groupby('state_id')[d_cols].sum().T.merge(cal.set_index('d')['date'],left_index=True,right_index=True,validate='1:1').set_index('date')
df[df.index.str.startswith('2015-10-') | df.index.str.startswith('2015-11-')| df.index.str.startswith('2015-12-')].plot(kind='line',figsize=(18, 8))
plt.title("Time series] Total sales by region (October - December 2015)")
plt.show()

### 【時系列】地域ごとの総販売数 (2015年10月分)

In [ ]:
df = stv.groupby('state_id')[d_cols].sum().T.merge(cal.set_index('d')['date'],left_index=True,right_index=True,validate='1:1').set_index('date')
df = df[df.index.str.startswith('2015-10-')].plot(kind='line',figsize=(18, 8))

plt.title("Time series] Total sales by region (October 2015)")
plt.show()

## 4.地域毎の店舗数

In [ ]:
df= stv.groupby('state_id').nunique()['store_id'].sort_values()
df.name = 'state_sum'
pd.DataFrame(df).plot(kind='barh', figsize=(15, 5), title='Number of stores by region')
plt.show()

## 5. 地域ごとに商品カテゴリの販売数

In [ ]:
df = stv.groupby(['state_id','cat_id']).sum().sum(axis=1)
df = pd.DataFrame({ 'FOODS' : [df['CA','FOODS'], df['TX','FOODS'], df['WI','FOODS']],\
                   'HOBBIES' : [df['CA','HOBBIES'], df['TX','HOBBIES'], df['WI','HOBBIES']],\
                   'HOUSEHOLD' : [df['CA','HOUSEHOLD'], df['TX','HOUSEHOLD'], df['WI','HOUSEHOLD']]},\
                  index=['CA','TX','WI'])
df

In [ ]:
df.plot(kind='barh', alpha=0.6,figsize=(9, 3), title='Number of sales in each product category by region')
plt.show()

## 6. よく売れている商品TOP5

In [ ]:
stv.groupby(['item_id']).sum().sum(axis=1).sort_values(axis=0,ascending=False)

### 考察
- 「FOODS_3_090,FOODS_3_586,FOODS_3_252,FOODS_3_555,FOODS_3_714」がよく売れていることがわかる。


## 7.売れ筋商品　店舗毎の売り上げ状況

売れ筋商品である「FOODS_3_090,FOODS_3_586,FOODS_3_252,FOODS_3_555,FOODS_3_714」を対象にする。

In [ ]:
df = stv.groupby(['item_id','store_id']).sum().sum(axis=1)
df['FOODS_3_090'].plot(kind='barh', alpha=0.6,figsize=(9, 3), title='FOODS_3_090')
plt.show()

In [ ]:
df['FOODS_3_586'].plot(kind='barh', alpha=0.6,figsize=(9, 3), title='FOODS_3_586')
plt.show()

In [ ]:
df['FOODS_3_252'].plot(kind='barh', alpha=0.6,figsize=(9, 3), title='FOODS_3_252')
plt.show()

In [ ]:
df['FOODS_3_555'].plot(kind='barh', alpha=0.6,figsize=(9, 3), title='FOODS_3_555')
plt.show()

In [ ]:
df['FOODS_3_714'].plot(kind='barh', alpha=0.6,figsize=(9, 3), title='FOODS_3_714')
plt.show()